Example (Table 2) from [A clarification of the nuances in the fairness metrics landscape](https://arxiv.org/abs/2106.00467)

[Scientific Reports publication](https://www.nature.com/articles/s41598-022-07939-1)

In [ ]:
!pip install fairlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# General import 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# ML Model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Fairlearn algorithms and utils
from fairlearn.postprocessing import ThresholdOptimizer
# from fairlearn.reductions import GridSearch, EqualizedOdds

# Metrics
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference, equalized_odds_ratio)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import mutual_info_score, adjusted_mutual_info_score, normalized_mutual_info_score

# Create Dataset

### Synthetic

In [ ]:
def create_synth(dim = 15000, bln_y_dip_a=True, thr_supp=0.33):
    np.random.seed(42)
    # # # dataset
    # Random var
    N1 = np.random.normal(0, 0.5, dim)
    N2 = np.random.normal(0, 0.5, dim)
    N3 = np.random.binomial(1, 0.5, size=dim)
    Ny = np.random.normal(0, 0.5, dim)
    A = np.random.binomial(1, 0.5, size=dim)
    # X var
    k=1.5
    X1 = N1 + k*A
    X2 = N2
    X3 = 1*((N3+A)>=1)
    X = pd.DataFrame({'X1':X1, 'X2':X2, 'X3':X3, 'A':A})
    # Y var
    if bln_y_dip_a:
        y = X1+2*X2+Ny+4*A+X3/2
    else:
        y = X1+2*X2+Ny+X3/2
    thres = y.mean()
    y = pd.Series(1*(y>thres))
    # Split train test
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
    # individual set
    X_ind_train = X_train[[i for i in X_train.columns if i!='A']]
    X_ind_test = X_test[[i for i in X_test.columns if i!='A']]
    # suppression set
    X_supp_train = X_train[[i for i in X_train.columns if i!='A' and 
                            abs(np.corrcoef(X_train[i], X_train['A'])[0,1])<thr_supp]]
    X_supp_test = X_test[[i for i in X_test.columns if i!='A' and 
                          abs(np.corrcoef(X_train[i], X_train['A'])[0,1])<thr_supp]]

    return X_train, X_ind_train, X_supp_train, X_test, X_ind_test, X_supp_test, y_train, y_test

In [ ]:
# thr_supp=0.66 #0.05, 0.66
# X_train, X_ind_train, X_supp_train, X_test, X_ind_test, X_supp_test, y_train, y_test = create_synth(bln_y_dip_a=True, thr_supp=thr_supp)
# sens_var = 'A'
# cond_var = 'X3'

### Adult

https://archive.ics.uci.edu/ml/datasets/adult

Load this dataset https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data in Colab folder before executing the next cells

In [ ]:
pd.read_csv('adult.data', header=None).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
# Code modified from https://github.com/FLE-ISP/CEILS/blob/main/experiments_run/adult_income_experiments/run_experiment.py
def adult(thr_supp=0.05):
  dataset = pd.read_csv('adult.data', header=None)
  del dataset[0]
  dataset = dataset.dropna()
  X = dataset.iloc[:, 0:13]
  X.columns = [str(i) for i in X.columns]
  # column 9: gender (A)
  # column 5: status (for CDP)

  for i in X.columns:
    if X[i].dtypes == 'O':
      le = preprocessing.LabelEncoder()
      le.fit(X[i])
      X.loc[:, i] = le.transform(X[i])
    else:
      # normalise
      X.loc[:, i] = X.loc[:, i]/max(X.loc[:, i])

  y = dataset.iloc[:, 13]
  le = preprocessing.LabelEncoder()
  le.fit(y)
  y = pd.Series(le.transform(y))

  # Split train test
  X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.33, random_state=42)
  # individual set
  X_ind_train = X_train[[i for i in X_train.columns if i!='9']]
  X_ind_test = X_test[[i for i in X_test.columns if i!='9']]
  # suppression set
  X_supp_train = X_train[[i for i in X_train.columns if i!='9' and 
                          abs(np.corrcoef(X_train[i], X_train['9'])[0,1])<thr_supp]]
  X_supp_test = X_test[[i for i in X_test.columns if i!='9' and 
                        abs(np.corrcoef(X_train[i], X_train['9'])[0,1])<thr_supp]]

  return X_train, X_ind_train, X_supp_train, X_test, X_ind_test, X_supp_test, y_train, y_test

In [ ]:
# thr_supp=0.10 # 0.05, 0.10
# X_train, X_ind_train, X_supp_train, X_test, X_ind_test, X_supp_test, y_train, y_test = adult(thr_supp=thr_supp)
# sens_var = '9'
# cond_var = '5'

# Fit model

In [ ]:
def fit_models(X_train, X_ind_train, X_supp_train, y_train):

  # Fit classifier
  model = RandomForestClassifier
  clf = model(random_state=0,).fit(X_train, y_train)
  clf_ind = model(random_state=0, ).fit(X_ind_train, y_train)
  clf_supp = model(random_state=0,).fit(X_supp_train, y_train)

  # KNN Individual metric
  neigh = KNeighborsClassifier(n_neighbors=5, weights='distance')
  neigh.fit(X_ind_train, y_train)
  neigh_supp = KNeighborsClassifier(n_neighbors=5, weights='distance')
  neigh_supp.fit(X_supp_train, y_train)

  return clf, clf_ind, clf_supp, neigh, neigh_supp

# Fairlearn
Code modified from https://github.com/fairlearn/fairlearn/blob/main/notebooks/Binary%20Classification%20with%20the%20UCI%20Credit-card%20Default%20Dataset.ipynb

In [ ]:
# Helper functions
def get_metrics_df(models_dict, y_true, group, X_ind_test=None, X_supp_test=None, dct_flip=None, neigh=None, neigh_supp=None):
    metrics_dict = {
        # "Overall selection rate": (
        #     lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        # "Overall balanced error rate": (
        #     lambda x: 1-balanced_accuracy_score(y_true, x), True),
        # "Balanced error rate difference": (
        #     lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        # " ------": (lambda x: "", True),
        # "False positive rate difference": (
        #     lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
        # "False negative rate difference": (
        #     lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds ratio": (
            lambda x: equalized_odds_ratio(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "ACC score": (
            lambda x: accuracy_score(y_true, 1*(x>0.5)), False),
        "F1 score": (
            lambda x: f1_score(y_true, 1*(x>0.5)), False),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "ACC score difference": (
            lambda x: MetricFrame(metrics=accuracy_score, y_true=y_true, y_pred=1*(x>0.5), sensitive_features=group).difference(method='between_groups'), False),  
        "F1 score difference": (
            lambda x: MetricFrame(metrics=f1_score, y_true=y_true, y_pred=1*(x>0.5), sensitive_features=group).difference(method='between_groups'), False),
        "AUC difference": (
            lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),  
        "   ------": (lambda x: "", True),
        "Flip": (lambda x: dct_flip[x], 'no'),
        "Individuality": (lambda x: neigh.score(X_ind_test, x), True),
        "Individuality suppression": (lambda x: neigh_supp.score(X_supp_test, x), True),
        "    ------": (lambda x: "", True),
        "Mutual information A, y_pred": (lambda x: normalized_mutual_info_score(group, x), True),
        "Mutual information A, y_true": (lambda x: normalized_mutual_info_score(group, y_true), True),
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
            list_tmp = []
            for model_name, (preds, scores) in models_dict.items():
                try:
                    if use_preds==True:
                        list_tmp.append(metric_func(preds))
                    elif use_preds==False:
                        list_tmp.append(metric_func(scores))
                    else:
                        list_tmp.append(metric_func(model_name))
                except:
                    print('problem in: ', metric_name)
                    list_tmp.append('NaN')
            df_dict[metric_name] = list_tmp
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

### Mitigation

In [ ]:
def mitigations(X_train, X_test, X_ind_test, X_supp_test, y_train,
                thr_supp, sens_var, cond_var,
                clf, clf_ind, clf_supp
                ):
  X_flip = X_test.copy()
  X_flip[sens_var] = 1-X_flip[sens_var]
  dct_flip={'FTU':1, 'Suppression_'+str(thr_supp):1}

  # # # unmitigated
  dct_flip['Unmitigated'] = 1-abs(clf.predict(X_test) - clf.predict(X_flip)).mean()

  # # # equalized_odds
  postprocess_est_eo = ThresholdOptimizer(estimator=clf, 
      constraints="equalized_odds", predict_method='predict_proba', prefit=True)
  postprocess_est_eo.fit(X_train, y_train, sensitive_features=X_train[sens_var])
  postprocess_preds_eo = postprocess_est_eo.predict(X_test, sensitive_features=X_test[sens_var])
  postprocess_preds_eo_flip = postprocess_est_eo.predict(X_flip, sensitive_features=X_flip[sens_var])
  dct_flip['EO'] = 1-abs(postprocess_preds_eo-postprocess_preds_eo_flip).mean()
  # # # demographic_parity
  postprocess_est_dp = ThresholdOptimizer(estimator=clf, 
    constraints="demographic_parity", predict_method='predict_proba', prefit=True)
  postprocess_est_dp.fit(X_train, y_train, sensitive_features=X_train[sens_var])
  postprocess_preds_dp = postprocess_est_dp.predict(X_test, sensitive_features=X_test[sens_var])
  postprocess_preds_dp_flip = postprocess_est_dp.predict(X_flip, sensitive_features=X_flip[sens_var])
  dct_flip['DP'] = 1-abs(postprocess_preds_dp-postprocess_preds_dp_flip).mean()
  # # # Conditional demographic_parity
  postprocess_preds_cdp = 0*postprocess_preds_dp.copy()
  postprocess_preds_cdp_flip = 0*postprocess_preds_dp.copy()
  for i in X_train[cond_var].unique():
      postprocess_est_dp_0 = ThresholdOptimizer(estimator=clf, 
        constraints="demographic_parity", predict_method='predict_proba', prefit=True)
      postprocess_est_dp_0.fit(X_train.loc[X_train[cond_var]==i], y_train[X_train[cond_var]==i], 
                              sensitive_features=X_train.loc[X_train[cond_var]==i, sens_var])
      postprocess_preds_dp_0 = postprocess_est_dp_0.predict(X_test.loc[X_test[cond_var]==i],
                                sensitive_features=X_test.loc[X_test[cond_var]==i, sens_var])
      postprocess_preds_dp_0_flip = postprocess_est_dp_0.predict(X_flip.loc[X_flip[cond_var]==i],
                                sensitive_features=X_flip.loc[X_flip[cond_var]==i, sens_var])
      # Define the prediction with the same dimensionality of test prediction
      postprocess_preds_cdp[X_test[cond_var]==i] = postprocess_preds_dp_0
      postprocess_preds_cdp_flip[X_test[cond_var]==i] = postprocess_preds_dp_0_flip
  dct_flip['CDP'] = 1-abs(postprocess_preds_cdp-postprocess_preds_cdp_flip).mean()

  # # # Build dictionary of mitigated prediction
  models_dict = {"Unmitigated": (clf.predict(X_test), clf.predict_proba(X_test)[:, 1]),
               "FTU":  (clf_ind.predict(X_ind_test), clf_ind.predict_proba(X_ind_test)[:, 1]),
               "Suppression_"+str(thr_supp): (clf_supp.predict(X_supp_test), clf_supp.predict_proba(X_supp_test)[:, 1]),
              "EO": (postprocess_preds_eo, postprocess_preds_eo),
               "DP": (postprocess_preds_dp, postprocess_preds_dp),
               "CDP": (postprocess_preds_cdp, postprocess_preds_cdp)}

  return models_dict, dct_flip

In [ ]:
def pipeline(func_data_gen, param_dict, sens_var, cond_var):
  """Pipeline that create a synthetic dataset, fit the models, mitigate those
   and output the results metrics

  Parameters
  ----------
  func_data_gen: function
      Method for data generation. E.g. 'create_synth' or 'adult'
  param_dict : dict
      Dictonary for setting the dataset creation
  sens_var : str
      Name of the sensitive variable. E.g. 'gender' or '9' or 'A'
  cond_var : str
      Name of the variable to condition used by the Conditional Demographic Parity.
      E.g. 'job' or '5' or 'X3'

  Returns
  -------
  pd.DataFrame
      a DataFrame of the output results: metrics per each model.
  """
  print("The parameters for data generetion are: ", param_dict, '\n')
  thr_supp = param_dict["thr_supp"]
  # Create dataset
  print("Start creation dataset.", '\n')
  X_train, X_ind_train, X_supp_train, X_test, X_ind_test, X_supp_test, y_train, y_test = func_data_gen(**param_dict)
  df_total = X_train.copy()
  df_total['Y'] = y_train
  print("The correlation matrix is: ", '\n', df_total.corr(), '\n',
        "The value counts is: ", '\n', df_total[sens_var].value_counts(), '\n')
  # Fit models
  print("Fitting models.", '\n')
  clf, clf_ind, clf_supp, neigh, neigh_supp = fit_models(X_train, X_ind_train, X_supp_train, y_train)
  # Mitigate models
  print("Mitigate models.", '\n')
  models_dict, dct_flip = \
    mitigations(X_train, X_test, X_ind_test, X_supp_test, y_train,
                thr_supp, sens_var, cond_var,
                clf, clf_ind, clf_supp)
  # return summary table
  print("Report output.", '\n')
  return get_metrics_df(models_dict, y_test, X_test[sens_var], X_ind_test, X_supp_test, dct_flip, neigh, neigh_supp)

## Adult

In [ ]:
# Adult
# thr_supp=0.05
param_dict = {"thr_supp": 0.05}
dtf_res = pipeline(adult, param_dict, sens_var="9", cond_var="5")
dtf_res

The parameters for data generetion are:  {'thr_supp': 0.05} 

Start creation dataset. 

The correlation matrix is:  
            1         2         3         4         5         6         7  \
1   1.000000 -0.009363  0.032804  0.058294 -0.064130  0.256130 -0.090961   
2  -0.009363  1.000000 -0.033407 -0.043347  0.030903  0.008778  0.010406   
3   0.032804 -0.033407  1.000000  0.359926 -0.035444 -0.023089 -0.014653   
4   0.058294 -0.043347  0.359926  1.000000 -0.070465  0.104245 -0.100073   
5  -0.064130  0.030903 -0.035444 -0.070465  1.000000 -0.004708  0.189394   
6   0.256130  0.008778 -0.023089  0.104245 -0.004708  1.000000 -0.078261   
7  -0.090961  0.010406 -0.014653 -0.100073  0.189394 -0.078261  1.000000   
8   0.052765 -0.019986  0.016394  0.034089 -0.068426  0.005799 -0.114718   
9   0.093014  0.024001 -0.020697  0.020041 -0.127057  0.078091 -0.582060   
10  0.030524  0.006266  0.030819  0.125227 -0.047165  0.025853 -0.054258   
11  0.012200 -0.012383  0.021294  0.088854 -0.

,Unmitigated,FTU,Suppression_0.05,EO,DP,CDP
Demographic parity difference,0.187774,0.183434,0.068082,0.213377,0.036145,0.164281
Demographic parity ratio,0.313471,0.328315,0.735307,0.301685,0.886442,0.381728
------,,,,,,
Equalized odds ratio,0.271277,0.304753,0.89122,0.245539,0.477292,0.4719
Equalized odds difference,0.090974,0.086832,0.054715,0.112626,0.139192,0.116279
------,,,,,,
ACC score,0.842639,0.83994,0.762982,0.83659,0.78029,0.831472
F1 score,0.649097,0.643892,0.497732,0.653785,0.586587,0.624196
Overall AUC,0.884671,0.884344,0.734089,0.772389,0.737777,0.748233
ACC score difference,0.118566,0.116734,0.055343,0.126365,0.065805,0.097529


In [ ]:
# Adult
# thr_supp=0.10
param_dict = {"thr_supp": 0.10}
dtf_res = pipeline(adult, param_dict, sens_var="9", cond_var="5")
dtf_res

The parameters for data generetion are:  {'thr_supp': 0.1} 

Start creation dataset. 

The correlation matrix is:  
            1         2         3         4         5         6         7  \
1   1.000000 -0.009363  0.032804  0.058294 -0.064130  0.256130 -0.090961   
2  -0.009363  1.000000 -0.033407 -0.043347  0.030903  0.008778  0.010406   
3   0.032804 -0.033407  1.000000  0.359926 -0.035444 -0.023089 -0.014653   
4   0.058294 -0.043347  0.359926  1.000000 -0.070465  0.104245 -0.100073   
5  -0.064130  0.030903 -0.035444 -0.070465  1.000000 -0.004708  0.189394   
6   0.256130  0.008778 -0.023089  0.104245 -0.004708  1.000000 -0.078261   
7  -0.090961  0.010406 -0.014653 -0.100073  0.189394 -0.078261  1.000000   
8   0.052765 -0.019986  0.016394  0.034089 -0.068426  0.005799 -0.114718   
9   0.093014  0.024001 -0.020697  0.020041 -0.127057  0.078091 -0.582060   
10  0.030524  0.006266  0.030819  0.125227 -0.047165  0.025853 -0.054258   
11  0.012200 -0.012383  0.021294  0.088854 -0.0

,Unmitigated,FTU,Suppression_0.1,EO,DP,CDP
Demographic parity difference,0.187774,0.183434,0.091106,0.213936,0.041602,0.160511
Demographic parity ratio,0.313471,0.328315,0.63307,0.300494,0.871001,0.390482
------,,,,,,
Equalized odds ratio,0.271277,0.304753,0.856591,0.243114,0.465924,0.470845
Equalized odds difference,0.090974,0.086832,0.073718,0.11314,0.145231,0.090105
------,,,,,,
ACC score,0.842639,0.83994,0.783454,0.836683,0.777871,0.833333
F1 score,0.649097,0.643892,0.524423,0.654051,0.582473,0.627418
Overall AUC,0.884671,0.884344,0.767029,0.772585,0.734841,0.749993
ACC score difference,0.118566,0.116734,0.069998,0.126645,0.070992,0.100196


## Synthetic

### Y depend on A

In [ ]:
# Synthetic Y high dependence on A
# thr_supp=0.05
param_dict = {"bln_y_dip_a":True, "thr_supp": 0.05}
dtf_res = pipeline(create_synth, param_dict, sens_var="A", cond_var="X3")
dtf_res

The parameters for data generetion are:  {'bln_y_dip_a': True, 'thr_supp': 0.05} 

Start creation dataset. 

The correlation matrix is:  
           X1        X2        X3         A         Y
X1  1.000000 -0.014188  0.482586  0.831844  0.828682
X2 -0.014188  1.000000 -0.004034 -0.007380  0.031672
X3  0.482586 -0.004034  1.000000  0.580846  0.575283
A   0.831844 -0.007380  0.580846  1.000000  0.981284
Y   0.828682  0.031672  0.575283  0.981284  1.000000 
 The value counts is:  
 1    5138
0    4912
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 



,Unmitigated,FTU,Suppression_0.05,EO,DP,CDP
Demographic parity difference,0.989102,0.885228,0.003139,0.986269,0.002366,0.509005
Demographic parity ratio,0.006869,0.057821,0.993881,0.007693,0.99762,0.487878
------,,,,,,
Equalized odds ratio,0.005243,0.132049,0.652116,0.006196,0.52831,0.80164
Equalized odds difference,0.615802,0.394354,0.238543,0.520564,0.467671,0.197954
------,,,,,,
ACC score,0.993131,0.94404,0.501818,0.993333,0.50303,0.755758
F1 score,0.993112,0.943642,0.506405,0.99331,0.666576,0.802419
Overall AUC,0.998515,0.984415,0.504314,0.993338,0.50522,0.756822
ACC score difference,0.001561,0.004897,0.021833,0.00035,0.97701,0.47359


In [ ]:
# Synthetic Y high dependence on A
# thr_supp=0.66
param_dict = {"bln_y_dip_a":True, "thr_supp": 0.66}
dtf_res = pipeline(create_synth, param_dict, sens_var="A", cond_var="X3")
dtf_res

The parameters for data generetion are:  {'bln_y_dip_a': True, 'thr_supp': 0.66} 

Start creation dataset. 

The correlation matrix is:  
           X1        X2        X3         A         Y
X1  1.000000 -0.014188  0.482586  0.831844  0.828682
X2 -0.014188  1.000000 -0.004034 -0.007380  0.031672
X3  0.482586 -0.004034  1.000000  0.580846  0.575283
A   0.831844 -0.007380  0.580846  1.000000  0.981284
Y   0.828682  0.031672  0.575283  0.981284  1.000000 
 The value counts is:  
 1    5138
0    4912
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 



,Unmitigated,FTU,Suppression_0.66,EO,DP,CDP
Demographic parity difference,0.989102,0.885228,0.360052,0.986269,0.002366,0.509005
Demographic parity ratio,0.006869,0.057821,0.475457,0.007693,0.99762,0.487878
------,,,,,,
Equalized odds ratio,0.005243,0.132049,0.870267,0.006196,0.52831,0.80164
Equalized odds difference,0.615802,0.394354,0.089444,0.520564,0.467671,0.197954
------,,,,,,
ACC score,0.993131,0.94404,0.682222,0.993333,0.50303,0.755758
F1 score,0.993112,0.943642,0.68331,0.99331,0.666576,0.802419
Overall AUC,0.998515,0.984415,0.759242,0.993338,0.50522,0.756822
ACC score difference,0.001561,0.004897,0.013998,0.00035,0.97701,0.47359


### Y doesn't depend on A

In [ ]:
# Synthetic Y low dependence on A
# thr_supp=0.05
param_dict = {"bln_y_dip_a":False, "thr_supp": 0.05}
dtf_res = pipeline(create_synth, param_dict, sens_var="A", cond_var="X3")
dtf_res

The parameters for data generetion are:  {'bln_y_dip_a': False, 'thr_supp': 0.05} 

Start creation dataset. 

The correlation matrix is:  
           X1        X2        X3         A         Y
X1  1.000000 -0.014188  0.482586  0.831844  0.566677
X2 -0.014188  1.000000 -0.004034 -0.007380  0.502406
X3  0.482586 -0.004034  1.000000  0.580846  0.364137
A   0.831844 -0.007380  0.580846  1.000000  0.516961
Y   0.566677  0.502406  0.364137  0.516961  1.000000 
 The value counts is:  
 1    5138
0    4912
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 



,Unmitigated,FTU,Suppression_0.05,EO,DP,CDP
Demographic parity difference,0.547445,0.549031,0.002027,0.538133,0.006736,0.294673
Demographic parity ratio,0.289014,0.284314,0.995961,0.294791,0.99125,0.613839
------,,,,,,
Equalized odds ratio,0.269587,0.261603,0.519817,0.294946,0.336817,0.706129
Equalized odds difference,0.216602,0.219465,0.201089,0.207779,0.478849,0.101212
------,,,,,,
ACC score,0.881414,0.883838,0.630101,0.88202,0.657374,0.795354
F1 score,0.880618,0.882629,0.629727,0.880962,0.729073,0.816186
Overall AUC,0.952006,0.95245,0.712272,0.882,0.658345,0.795776
ACC score difference,0.005911,0.005122,0.012769,0.007128,0.454612,0.179763


In [ ]:
# Synthetic Y low dependence on A
# thr_supp=0.66
param_dict = {"bln_y_dip_a":False, "thr_supp": 0.66}
dtf_res = pipeline(create_synth, param_dict, sens_var="A", cond_var="X3")
dtf_res

The parameters for data generetion are:  {'bln_y_dip_a': False, 'thr_supp': 0.66} 

Start creation dataset. 

The correlation matrix is:  
           X1        X2        X3         A         Y
X1  1.000000 -0.014188  0.482586  0.831844  0.566677
X2 -0.014188  1.000000 -0.004034 -0.007380  0.502406
X3  0.482586 -0.004034  1.000000  0.580846  0.364137
A   0.831844 -0.007380  0.580846  1.000000  0.516961
Y   0.566677  0.502406  0.364137  0.516961  1.000000 
 The value counts is:  
 1    5138
0    4912
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 



,Unmitigated,FTU,Suppression_0.66,EO,DP,CDP
Demographic parity difference,0.547445,0.549031,0.231133,0.538133,0.006736,0.294673
Demographic parity ratio,0.289014,0.284314,0.626153,0.294791,0.99125,0.613839
------,,,,,,
Equalized odds ratio,0.269587,0.261603,0.954095,0.294946,0.336817,0.706129
Equalized odds difference,0.216602,0.219465,0.023967,0.207779,0.478849,0.101212
------,,,,,,
ACC score,0.881414,0.883838,0.72404,0.88202,0.657374,0.795354
F1 score,0.880618,0.882629,0.724152,0.880962,0.729073,0.816186
Overall AUC,0.952006,0.95245,0.816964,0.882,0.658345,0.795776
ACC score difference,0.005911,0.005122,0.010311,0.007128,0.454612,0.179763
